In [50]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [1000]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S2'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23c
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

for SNR in SNR_list:
    ber_his = []
    dfe_ber_his = []
    LS_ber_his = []
    comb_ber_his = []
    sta_ber_his = []
    for i in range(packet_num):
        rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
                N_reservoir=16,
                spectral_radius=0.2,
                sparsity=0.4,
                noise=1e-6,
                lut_activation=False,  # True,
                tanh_lut=tanh_lut,
                input_scale=25,  #40, #50, # 25,
                reservoir_input_scale = 8,  #4,  #5,
                show_wout=False,
                output_folder= output_folder,
                debug=debug,
                use_fpga= None,
                sock= None,  # usock
                addr = None) # addr

        train_input, train_label, test_input, test_label = rc.run()
        print(train_input.shape, train_label.shape, test_input.shape, test_label.shape)
        print(train_input[0:20, :])


(7521, 4) (7521, 2) (7521, 4) (89, 64)
[[ 0.00000000e+00  0.00000000e+00  5.33991188e-03  3.81591271e-03]
 [ 5.33991188e-03  3.81591271e-03 -9.15646306e-03  2.28503959e-03]
 [-9.15646306e-03  2.28503959e-03 -8.38822816e-03 -6.87190186e-03]
 [-8.38822816e-03 -6.87190186e-03  1.37370880e-02 -4.56624148e-03]
 [ 1.37370880e-02 -4.56624148e-03  9.92005662e-03 -1.51552640e-03]
 [ 9.92005662e-03 -1.51552640e-03  1.44981241e-02 -1.50769347e-03]
 [ 1.44981241e-02 -1.50769347e-03  3.06414004e-03 -8.38806227e-03]
 [ 3.06414004e-03 -8.38806227e-03 -1.44923601e-02 -2.31317008e-03]
 [-1.44923601e-02 -2.31317008e-03  7.51448215e-04  6.10509742e-03]
 [ 7.51448215e-04  6.10509742e-03  2.31284830e-03 -1.14395603e-02]
 [ 2.31284830e-03 -1.14395603e-02 -8.37669868e-03 -6.88595173e-03]
 [-8.37669868e-03 -6.88595173e-03 -3.08635244e-03  1.37255396e-02]
 [-3.08635244e-03  1.37255396e-02  7.33863544e-04  1.06834610e-02]
 [ 7.33863544e-04  1.06834610e-02  7.22673667e-04  1.37354349e-02]
 [ 7.22673667e-04  1.37

(7521, 4) (7521, 2) (7521, 4) (89, 64)
[[ 0.          0.         -0.00457888 -0.00457663]
 [-0.00457888 -0.00457663  0.00991312 -0.01449987]
 [ 0.00991312 -0.01449987  0.00382042  0.0114425 ]
 [ 0.00382042  0.0114425  -0.00380588  0.01449856]
 [-0.00380588  0.01449856 -0.00456762  0.01373663]
 [-0.00456762  0.01373663  0.00077609  0.01525851]
 [ 0.00077609  0.01525851  0.01143387 -0.01069267]
 [ 0.01143387 -0.01069267 -0.00382576 -0.00991424]
 [-0.00382576 -0.00991424  0.00077234  0.00762864]
 [ 0.00077234  0.00762864  0.01754597 -0.00154964]
 [ 0.01754597 -0.00154964 -0.00764421 -0.0099072 ]
 [-0.00764421 -0.0099072  -0.01526647 -0.00455336]
 [-0.01526647 -0.00455336 -0.01373849 -0.0030282 ]
 [-0.01373849 -0.0030282  -0.0122242  -0.00913298]
 [-0.0122242  -0.00913298  0.01448719 -0.00460626]
 [ 0.01448719 -0.00460626  0.00688256  0.00761522]
 [ 0.00688256  0.00761522 -0.00764139 -0.00532382]
 [-0.00764139 -0.00532382  0.00988636 -0.01375639]
 [ 0.00988636 -0.01375639  0.00384104  0.01

(7521, 4) (7521, 2) (7521, 4) (89, 64)
[[ 0.00000000e+00  0.00000000e+00 -3.05119956e-03 -5.34107421e-03]
 [-3.05119956e-03 -5.34107421e-03  1.67850929e-02  2.99544354e-06]
 [ 1.67850929e-02  2.99544354e-06 -2.29105462e-03  9.15496057e-03]
 [-2.29105462e-03  9.15496057e-03 -1.67859995e-02  3.04684218e-03]
 [-1.67859995e-02  3.04684218e-03 -1.29702981e-02 -4.62932803e-06]
 [-1.29702981e-02 -4.62932803e-06 -1.67869977e-02  4.57076239e-03]
 [-1.67869977e-02  4.57076239e-03  6.86227067e-03  9.15877099e-03]
 [ 6.86227067e-03  9.15877099e-03  1.37348863e-02 -3.04448222e-03]
 [ 1.37348863e-02 -3.04448222e-03 -6.86526546e-03 -2.29296074e-03]
 [-6.86526546e-03 -2.29296074e-03  3.04134992e-03  1.60241281e-02]
 [ 3.04134992e-03  1.60241281e-02  9.15713608e-03 -2.28234003e-03]
 [ 9.15713608e-03 -2.28234003e-03  1.41603095e-05 -1.83110053e-02]
 [ 1.41603095e-05 -1.83110053e-02 -7.52792130e-04 -1.22079713e-02]
 [-7.52792130e-04 -1.22079713e-02  1.54089412e-03 -1.67837265e-02]
 [ 1.54089412e-03 -1.67

(7521, 4) (7521, 2) (7521, 4) (89, 64)
[[ 0.          0.         -0.00534094  0.00381447]
 [-0.00534094  0.00381447 -0.00304988 -0.0160225 ]
 [-0.00304988 -0.0160225   0.00839269 -0.00076142]
 [ 0.00839269 -0.00076142  0.00457366  0.01678621]
 [ 0.00457366  0.01678621  0.0015222   0.0122078 ]
 [ 0.0015222   0.0122078   0.00609781  0.01602436]
 [ 0.00609781  0.01602436  0.00915876 -0.00762568]
 [ 0.00915876 -0.00762568 -0.00457106 -0.01373549]
 [-0.00457106 -0.01373549 -0.00229264  0.00686537]
 [-0.00229264  0.00686537  0.01602538 -0.00533095]
 [ 0.01602538 -0.00533095 -0.0030457  -0.00915755]
 [-0.0030457  -0.00915755 -0.0167862   0.00151364]
 [-0.0167862   0.00151364 -0.01220915  0.00227921]
 [-0.01220915  0.00227921 -0.0175487   0.00074799]
 [-0.0175487   0.00074799  0.00229794 -0.00991637]
 [ 0.00229794 -0.00991637  0.01449695 -0.00074883]
 [ 0.01449695 -0.00074883 -0.00534466  0.00380926]
 [-0.00534466  0.00380926 -0.0022713  -0.01602463]
 [-0.0022713  -0.01602463  0.00839342 -0.00